In [154]:
from collections import Counter
import re
from PIL import Image, ImageEnhance
import unidecode
 
def remove_accent(text):
    return unidecode.unidecode(text)


def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    # print('Texts:')
    list_t = []
    for text in texts:
        # try:
        #     text = float(text.description))
        # except:
        text = text.description
        list_t.append(text)
        #print('\n"{}"'.format(text.description))

        # vertices = (['({},{})'.format(vertex.x, vertex.y)
        #             for vertex in text.bounding_poly.vertices])

        # print('bounds: {}'.format(','.join(vertices)))

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    return list_t

def replace_invalid(list_detected_text_2):
    f = lambda x: x.replace(" ", ",", ":", ";", "!", "-", "=", "+")
    invalid_chars = [' ', ',', ';', '{', '}', '(', ')', '\n', '\t', '=', ":", ";", "!", "-", "=", "+","/", "_"]
    invalid_table = str.maketrans({k:'.' for k in invalid_chars})
    list_valid = [float(a.translate(invalid_table)) for a in list_detected_text_2]
    return list_valid

In [197]:
def get_info(path_p):
    temp2 = enhence_resolution(path_p,1.5)
    temp = detect_text(temp2)
    uni_text = remove_accent(temp[0])
    dict_info = {}
    # Define index
    try: 
        i1 = uni_text.index("a)")
    except:
        i1 = uni_text.index("dat so")
    i2 = uni_text.index(("do so" or "do"))
    try: 
        i3 = uni_text.index("Dia chi")
    except:
        i3 = uni_text.index("b)")
    try:
        i4 = uni_text.index("Dien tich")
    except:
        i4 =  uni_text.index("c)")
    try:
        i5 = uni_text.index("d)")
    except:
        i5 = uni_text.index("Hinh thuc")
    #find thua dat so
    try:
        if i1 < i2:
            num1 = re.findall(r"\d+",uni_text[i1:i2])[0]
        else:
            num1 = re.findall(r"\d+",uni_text[i1:])[0]
    except:
        num1 = -1
    dict_info["Thửa đất số"] = num1
    #find so to
    try:
        if i2 < i3:
            num2 = re.findall(r"\d+",uni_text[i2:i3])[0]
        else:
            num2 = re.findall(r"\d+",uni_text[i2:])[0]
    except:
        num2 = -1
    dict_info["Tờ bản đồ số"] = num2
    #find address
    try:
        if i3 < i4:
            add = temp[0][i3:i4].split(":")[1]
        else:
            add = temp[0][i3:].split(":")[1]
    except:
        add = "None"
    add = add.replace("\n", ",")
    dict_info["Địa chỉ thửa đất"] = add
    #Find dien tich
    try:
        # if i4 < i5:
        #     strs = re.findall(r'\d+[\W|_]?\d*', uni_text[i4:i5])
        #     num3 = replace_invalid(strs)[0]
        # else:
            strs = re.findall(r'\d+[.,_+-]?\d*', uni_text[i4:])
            num3 = replace_invalid(strs)[0]
            if num3 <= 10:
                
                num3 = replace_invalid(strs)[1]
            else:
                num3 = replace_invalid(strs)[0]
    except:
        num3 = -1
    dict_info["Diện tích"] = num3
    
    return dict_info

In [198]:
# temp = detect_text("info_field/information_22.jpg")
def enhence_resolution(img, p_factor):
    im = Image.open(img)
    enhancer = ImageEnhance.Contrast(im)
    factor = p_factor #gives original image
    im_output = enhancer.enhance(factor)
    name = 'original-image.png'
    im_output.save(name)
    return name

# detect_text(temp2)
# uni_text = remove_accent(temp[0])
# i4 = uni_text.index("Dien tich")
# strs = re.findall(r'\d+[.,_]?\d*', uni_text[i4:])
# strs
# float("1,3")
# get_info(temp2)

In [41]:
# import os
# path = 'info_field/'
# files = os.listdir(path)
# i = 0
# for file in files:
#     os.rename(os.path.join(path, file), os.path.join(path, 'information_' + str(i) + '.jpg'))
#     i = i + 1

In [199]:
dict_result  = {}
path = 'info_field/'
files = os.listdir(path)
for file in files:
    try:
        path_t = path + file
        dict_result[file] = get_info(path_t)
    except:
        dict_result[file] = {'Thửa đất số': -1, 'Tờ bản đồ số': -1,'Địa chỉ thửa đất': 'None', 'Diện tích': -1}

In [200]:
# dict_result
import pandas as pd
df = pd.DataFrame()
df["name"] = dict_result.keys()
df["infomation"] = dict_result.values()
list_so_thua = [df["infomation"][i]['Thửa đất số'] for i in range(0,len(df["name"]))]
list_so_bd = [df["infomation"][i]['Tờ bản đồ số'] for i in range(0,len(df["name"]))]
list_diachi = [df["infomation"][i]['Địa chỉ thửa đất'] for i in range(0,len(df["name"]))]
list_dientich = [df["infomation"][i]['Diện tích'] for i in range(0,len(df["name"]))]
df["Số thửa đất"] = list_so_thua
df["Số bản đồ"] = list_so_bd
df["Địa chỉ"] = list_diachi
df["Diện tích"] = list_dientich
df.drop("infomation", 1, inplace = True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [201]:
df

,name,Số thửa đất,Số bản đồ,Địa chỉ,Diện tích
0,information_13.jpg,576,48,"Xã Nhuận Đức, Huyện Củ Chi, Thành phố Hồ Chí ...",119.4
1,information_7.jpg,646,58,"Xã Bình Mỹ, Huyện Củ Chi, Tp Hồ Chí Minh,c)",108.6
2,information_16.jpg,63,11,"1261/1/15 Lê Đức Thọ, phường 13, quận Gò Vấp,...",77.0
3,information_12.jpg,534,29,"Xã Tân Quí Tây, Huyện Bình Chánh,4.",100.0
4,information_42.jpg,587,19,"Phường Long Phước, Quận 9, Thành phố Hồ Chí M...",688.8
5,information_4.jpg,616,16,"255/12A Quốc lộ 13 cũ, KP3, P. Hiệp Bình Phướ...",110.7
6,information_18.jpg,601,36,"Xã Tân Thạnh Đông, Huyện Củ Chi, Thành phố Hồ...",220.0
7,information_6.jpg,583,38,"Phường Long Phước, Quận 9, Thành phố Hồ Chí M...",700.0
8,information_19.jpg,684,45,"Phường Phú Hữu, Quận 9, Thành phố Hồ Chí Minh...",53.0
9,information_36.jpg,331,25,"phường Tân Phong, quận 7, thành phố Hồ Chí Mi...",137.5


In [203]:
# 34, 24, 4
# "abc".remove("a")